In [59]:
%reload_ext autoreload
%autoreload 2

import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from hero_model import Model

#warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 200)

### Lidando com dados categoricos
##### Removemos a coluna hero_names e usamos one-hot-encoding nas features categoricas exceto por Gender, que substituimos manualmente os valores de male para 0 e female para 1

In [60]:
data = pd.read_csv('data/clean_data.csv')
data.head()

,hero_names,Gender,Eye color,Race,Hair color,Height,Publisher,Alignment,Weight,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,Danger Sense,Underwater breathing,Marksmanship,Weapons Master,Power Augmentation,Animal Attributes,Longevity,Intelligence,Super Strength,Cryokinesis,Telepathy,Energy Armor,Energy Blasts,Duplication,Size Changing,Density Control,Stamina,Astral Travel,Audio Control,Dexterity,Omnitrix,Super Speed,Possession,Animal Oriented Powers,Weapon-based Powers,Electrokinesis,Darkforce Manipulation,Death Touch,Teleportation,Enhanced Senses,Telekinesis,Energy Beams,Magic,Hyperkinesis,Jump,Clairvoyance,Dimensional Travel,Power Sense,Shapeshifting,Peak Human Condition,Immortality,Camouflage,Element Control,Phasing,Astral Projection,Electrical Transport,Fire Control,Projection,Summoning,Enhanced Memory,Reflexes,Invulnerability,Energy Constructs,Force Fields,Self-Sustenance,Anti-Gravity,Empathy,Power Nullifier,Radiation Control,Psionic Powers,Elasticity,Substance Secretion,Elemental Transmogrification,Technopath/Cyberpath,Photographic Reflexes,Seismic Power,Animation,Precognition,Mind Control,Fire Resistance,Power Absorption,Enhanced Hearing,Nova Force,Insanity,Hypnokinesis,Animal Control,Natural Armor,Intangibility,Enhanced Sight,Molecular Manipulation,Heat Generation,Adaptation,Gliding,Power Suit,Mind Blast,Probability Manipulation,Gravity Control,Regeneration,Light Control,Echolocation,Levitation,Toxin and Disease Control,Banish,Energy Manipulation,Heat Resistance,Natural Weapons,Time Travel,Enhanced Smell,Illusions,Thirstokinesis,Hair Manipulation,Illumination,Omnipotent,Cloaking,Changing Armor,Power Cosmic,Biokinesis,Water Control,Radiation Immunity,Vision - Telescopic,Toxin and Disease Resistance,Spatial Awareness,Energy Resistance,Telepathy Resistance,Molecular Combustion,Omnilingualism,Portal Creation,Magnetism,Mind Control Resistance,Plant Control,Sonar,Sonic Scream,Time Manipulation,Enhanced Touch,Magic Resistance,Invisibility,Sub-Mariner,Radiation Absorption,Intuitive aptitude,Vision - Microscopic,Melting,Wind Control,Super Breath,Wallcrawling,Vision - Night,Vision - Infrared,Grim Reaping,Matter Absorption,The Force,Resurrection,Terrakinesis,Vision - Heat,Vitakinesis,Radar Sense,Qwardian Power Ring,Weather Control,Vision - X-Ray,Vision - Thermal,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,good,441.0,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,good,65.0,True,True,False,False,True,True,False,False,False,False,True,True,True,False,False,True,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,

In [61]:
data.shape

(684, 176)

In [62]:
data.drop("hero_names", axis=1, inplace=True)

In [63]:
data = data.replace(['Male', 'Female'], [0,1])

In [64]:
data = pd.get_dummies(data)
data.shape

(684, 308)

In [65]:
types = set(data.dtypes)
types

{dtype('bool'), dtype('uint8'), dtype('int64'), dtype('float64')}

### Predições

In [69]:
model = Model(data, "Flight")
model.train_test()

Train:
F1 Score: 1.0

Test:
F1 Score: 0.5263157894736842
True negatives:  87
True positives:  30
False negatives:  37
False positives:  17


In [70]:
grid = model.find_best_parameters()

In [71]:
best_parameters = grid.best_params_
best_parameters

{'criterion': 'gini',
 'max_depth': 12,
 'min_samples_leaf': 1,
 'min_samples_split': 8}

In [72]:
grid.cv_results_['std_test_score']

array([0.0200887 , 0.03342531, 0.02142336, 0.02972617, 0.03856389,
       0.04879905, 0.0286969 , 0.0286969 , 0.0286969 , 0.04027836,
       0.05545163, 0.05965406, 0.04896832, 0.04896832, 0.04896832,
       0.04896832, 0.04029627, 0.04436616, 0.01204716, 0.02753931,
       0.00953288, 0.03239093, 0.02686255, 0.00619081, 0.01401296,
       0.01401296, 0.01401296, 0.05949282, 0.0336751 , 0.04257626,
       0.0555217 , 0.0555217 , 0.0555217 , 0.0555217 , 0.05374527,
       0.04099788, 0.01927746, 0.01826842, 0.02153019, 0.00654793,
       0.03004513, 0.03489556, 0.02121295, 0.02121295, 0.02121295,
       0.02569714, 0.04404503, 0.07482793, 0.06100625, 0.06100625,
       0.06100625, 0.06100625, 0.04673842, 0.04332521, 0.02462199,
       0.02313851, 0.02462121, 0.02360445, 0.01315424, 0.03112116,
       0.04030711, 0.04030711, 0.04030711, 0.04066746, 0.05457533,
       0.05168986, 0.07081094, 0.07081094, 0.07081094, 0.07081094,
       0.05546251, 0.04015098, 0.02325376, 0.01664861, 0.02934

In [73]:
grid.best_score_

0.6358604983305348

In [74]:
model.set_best_parameters(best_parameters)
model.train_test()

Train:
F1 Score: 0.8231046931407944

Test:
F1 Score: 0.5333333333333332
True negatives:  94
True positives:  28
False negatives:  39
False positives:  10


## Usando mais correlacionados

In [110]:
def most_correlated(data, num, label):
    corr = data.corr()
    corr = corr.abs()
    corr = corr.sort_values(label, ascending=False).head(num)[label]
    return list(corr.index)

In [120]:
most_corr = most_correlated(data, 60, 'Flight')

In [121]:
most_corr

['Flight',
 'Energy Blasts',
 'Force Fields',
 'Vision - X-Ray',
 'Energy Manipulation',
 'Energy Absorption',
 'Super Speed',
 'Invulnerability',
 'Energy Constructs',
 'Phasing',
 'Molecular Manipulation',
 'Super Strength',
 'Super Breath',
 'Vision - Heat',
 'Vision - Telescopic',
 'Vision - Microscopic',
 'Invisibility',
 'Energy Beams',
 'Teleportation',
 'Time Travel',
 'Intangibility',
 'Fire Control',
 'Weapons Master',
 'Telekinesis',
 'Enhanced Hearing',
 'Reality Warping',
 'Immortality',
 'Self-Sustenance',
 'Publisher_DC Comics',
 'Telepathy',
 'Gravity Control',
 'Levitation',
 'Enhanced Senses',
 'Race_Kryptonian',
 'Stealth',
 'Time Manipulation',
 'Accelerated Healing',
 'Magnetism',
 'Lantern Power Ring',
 'Astral Projection',
 'Heat Generation',
 'Peak Human Condition',
 'Dimensional Awareness',
 'Longevity',
 'Gliding',
 'Race_God / Eternal',
 'Power Cosmic',
 'Vision - Thermal',
 'Race_Human',
 'Dimensional Travel',
 'Element Control',
 'Race_Cosmic Entity',
 'Ast

In [122]:
corr_model = Model(data[most_corr],'Flight') 

In [123]:
corr_model.train_test()

Train:
F1 Score: 0.9664429530201343

Test:
F1 Score: 0.5982905982905983
True negatives:  89
True positives:  35
False negatives:  32
False positives:  15


In [124]:
grid = corr_model.find_best_parameters()

In [125]:
grid.best_score_

0.586457953019769

In [126]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 12,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [127]:
corr_model.set_best_parameters(grid.best_params_)
corr_model.train_test()

Train:
F1 Score: 0.8409090909090909

Test:
F1 Score: 0.5794392523364486
True negatives:  95
True positives:  31
False negatives:  36
False positives:  9
